In [10]:
!pip install git+https://github.com/jsh9/python-plot-utilities

  Cloning https://github.com/jsh9/python-plot-utilities to c:\users\erika\appdata\local\temp\pip-req-build-uc2ycj6q
  Created wheel for plot-utils: filename=plot_utils-0.6.6-py3-none-any.whl size=13278151 sha256=340945e89e009354b79dfb0188c25b318c5f1c8f2864a438c2025ee0a5843dd8
  Stored in directory: C:\Users\erika\AppData\Local\Temp\pip-ephem-wheel-cache-6zff0_0x\wheels\d0\99\78\05b2d3be85d4e5d1f5fec6dcb8ac2b5866bbed0a86fd1756b0
Successfully built plot-utils


In [11]:
from brian2 import *
import sympy
import plot_utils

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [12]:
# Functions that are used in the notebook
from random import random, randrange

# Fetched from the brian2 totoruial
def visualise_connectivity(S):
    Ns = len(S.source)
    Nt = len(S.target)
    figure(figsize=(10, 4))
    subplot(121)
    plot(zeros(Ns), arange(Ns), 'ok', ms=10)
    plot(ones(Nt), arange(Nt), 'ok', ms=10)
    for i, j in zip(S.i, S.j):
        plot([0, 1], [i, j], '-k')
    xticks([0, 1], ['Source', 'Target'])
    ylabel('Neuron index')
    xlim(-0.1, 1.1) 
    ylim(-1, max(Ns, Nt))
    subplot(122)
    plot(S.i, S.j, 'ok')
    xlim(-1, Ns)
    ylim(-1, Nt)
    xlabel('Source neuron index')
    ylabel('Target neuron index')

# Generates data that we can test the network on.
def spikegen(N: int, end_time: int, star_time: int =  0, random_start: bool = False):
  delta_time = (end_time - star_time)
  # Initialize a direction list
  labels = [None] * delta_time
  spikes = [None] * delta_time
  times = [None] * delta_time

  if random_start:
    start_place = randrange(0, N)
  else:
    start_place = round(N/2)

  # Set first spike manually
  labels[0] = 0
  spikes[0] = start_place
  times[0] = 0
  
  t = 1
  id = start_place
  cooldown = 0
  target = id
  dir = 0

  while t < delta_time:
    # Get a target
    if id == target:
      while id == target:
        target = randrange(0, N)
      dir = round((target - id) / abs(target - id))
      #print(dir)

    # Check whether to move to stand still
    still = True
    if cooldown == 0:
      # 10% chance of standing still
      if randrange(0, 20) == 0:
        cooldown = 3
      else:
        still = False
    else:
      cooldown -= 1

    d = 0 if still == True else dir
    # Do movement
    id += d

    # Set output data
    labels[t] = d
    spikes[t] = id
    times[t] = t

    #print('t: {:1d}\tid: {:1d}\td: {:1d}\ttarget: {:1d}'.format(t, id, d, target), end='\n')
    t += 1

  return {
    'labels': labels,
    'spikes': spikes,
    'times': times
  }


def filter_spikes(data):
  ## Removes zeros so it dose not spike on them.
  z = list(zip(*[data[k] for k in data]))
  f = [r for r in z if r[0] != 0]
  v = [list(t) for t in zip(*f)]

  o = dict()
  for i, k in enumerate(data):
    o[k] = v[i]
  return o



def network_guess(left_spikes,right_spikes):
    ## Will return a list that display the movments
    left_i = 0
    left_last = len(left_spikes)
    right_i = 0
    right_last = len(right_spikes)
    total_lenght = left_last + right_last
    movments = []
    movments_i = 0
    
    while left_i + right_i < total_lenght:
        if left_spikes[left_i] == right_spikes[right_i]:
            right_i +=1
            left_i +=1
            movments.append(0)
        elif left_spikes[left_i] > right_spikes[right_i]:
            movments.append(1)
            right_i +=1
        else:
            movments.append(-1)
            left_i +=1
        if left_i == left_last:
            while right_i < right_last:
                movments.append(1)
                right_i +=1
        if right_i == right_last:
            while left_i < left_last:
                movments.append(-1)
                left_i +=1
    return movments

## Generate movment data

In [13]:
input_neurons = 100 ### insert no of input neurons here ###
time = 180

data = spikegen(input_neurons, time)
data_without_zeros = filter_spikes(data)
print(data['spikes'])
print(data['times'])

[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 67, 67, 67, 67, 68, 69, 70, 71, 72, 73, 74, 74, 74, 74, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 56, 56, 56, 56, 55, 54, 55, 56, 56, 56, 56, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 69, 68, 67, 66, 65, 64, 63, 63, 63, 63, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 46, 46, 46, 46, 46, 46, 46, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 31, 32, 33, 34, 34, 34, 34, 34, 35, 36, 37, 38, 39]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 

## Creates the network

In [14]:
#https://brian2.readthedocs.io/en/stable/examples/frompapers.Stimberg_et_al_2018.example_1_COBA.html
start_scope()

seed(11922)  # to get identical figures for repeated runs


################################################################################
# Model parameters
################################################################################
### General parameters

labels = data_without_zeros['labels']
spikes = data_without_zeros['spikes']
time_for_spikes = data_without_zeros['times']*ms

duration = time*ms    # Total simulation time
sim_dt = 0.01*ms        # Integrator/sampling step


### Neuron parameters
tau_m = 30*ms         # Excitatory synaptic time constant
V_th = 50e-3         # Firing threshold
V_r = 35e-3          # Reset potential       
R = 90e6




### Synapse parameters
we = 1          # Excitatory synaptic weight
wi = -1           # Inhibitory synaptic weight

#indices = [3,4,5,6,5,4,3,3,4,5,6,5,4,3] #np.linspace(0, 50, num=2)
#times = [10,11,12,13,14,15,16,30,31,32,33,34,35,36]*ms #np.linspace(10, 90, num=2)*ms




################################################################################
# Model definition
################################################################################
# Set the integration time (in this case not strictly necessary, since we are
# using the default value)
defaultclock.dt = sim_dt

### Neurons
neuron_eqs = '''
dv/dt = ((V_r-v)+R*I)/tau_m : 1
I : 1
'''
input_group = NeuronGroup(input_neurons, model=neuron_eqs,threshold='v>V_th', reset='v=V_r', method='euler')

neurons = NeuronGroup(3*input_neurons-2, model=neuron_eqs,threshold='v>V_th', reset='v=V_r', method='euler')

left = NeuronGroup(1, model=neuron_eqs,threshold='v>V_th',reset='v=V_r', method='euler')

right = NeuronGroup(1, model=neuron_eqs,threshold='v>V_th',reset='v=V_r', method='euler')

neurons.I  = 50e-12
neurons.v = input_group.v = left.v = right.v = 'V_r'

input_layer = SpikeGeneratorGroup(input_neurons, spikes, time_for_spikes, dt=sim_dt) # assigs 1/4 of the neurons to the input layer
#input_layer = input_group[:input_neurons]
inh_neurons = neurons[:input_neurons] # assigs 1/4 of the neurons to the inhibitory layer
left_layer  = neurons[1*input_neurons:2*input_neurons-1] # assigs 1/4 -1 of the neurons to the left exitatory layer
right_layer = neurons[2*input_neurons-1:3*input_neurons-2] # assigs 1/4 -1 of the neurons to the right exitatory layer
left_neuron = left[0] # assigns 1 of the neurons to the final left layer
right_neuron= right[0] # assigns 1 of the neurons to the final right layer

### Synapses
exc_syn = Synapses(input_layer, inh_neurons, model = 'w : 1', on_pre = 'v += w') # Connects the input layer to the inhibitory layer
exc_syn_left = Synapses(input_layer, left_layer, model = 'w : 1', on_pre = 'v += w') # Connects the input layer to the left exitatory layer
exc_syn_right = Synapses(input_layer, right_layer, model = 'w :1', on_pre = 'v += w') # Connects the input layer to the right exitatory layer
inh_syn_right = Synapses(inh_neurons, right_layer, model = 'w : 1', on_pre = 'v += w') # Connects the inhibitory layer to the right exitatory layer
inh_syn_left = Synapses(inh_neurons, left_layer, model = 'w : 1', on_pre = 'v += w') # Connects the inhibitory layer to the left exitatory layer
final_left = Synapses(left_layer, left_neuron, model = 'w : 1', on_pre = 'v += w') # Connects the left exitatory layer to the final left neuron
final_right = Synapses(right_layer, right_neuron, model = 'w : 1', on_pre = 'v += w') # Connects the right exitatory layer to the final right neuron

exc_syn.connect(i=[*range(0,input_neurons)],j=[*range(0,input_neurons)])
exc_syn_left.connect(i=[*range(1,input_neurons)],j=[*range(0,input_neurons-1)])
exc_syn_right.connect(i=[*range(0,input_neurons-1)],j=[*range(0,input_neurons-1)])
inh_syn_right.connect(i=[*range(1,input_neurons)],j=[*range(0,input_neurons-1)])
inh_syn_left.connect(i=[*range(0,input_neurons-1)],j=[*range(0,input_neurons-1)])
final_left.connect(i=[*range(0,input_neurons-1)],j=0)
final_right.connect(i=[*range(0,input_neurons-1)],j=0)

exc_syn.w = 60e-3
exc_syn_left.w = 60e-3
exc_syn_right.w = 60e-3
inh_syn_right.w = -100e-3
inh_syn_left.w = -100e-3
final_left.w = 60e-3
final_right.w = 60e-3


#visualise_connectivity(exc_syn)
#visualise_connectivity(exc_syn_left)
#visualise_connectivity(exc_syn_right)
#visualise_connectivity(inh_syn_left)
#visualise_connectivity(inh_syn_right)
#visualise_connectivity(final_left)
#visualise_connectivity(final_right)


# ##############################################################################
# # Monitors
# ##############################################################################
# Note that we could use a single monitor for all neurons instead, but in this
# way plotting is a bit easier in the end
#exc_mon = SpikeMonitor(input_layer)
#inh_mon = SpikeMonitor(inh_neurons)

### We record some additional data from a single excitatory neuron

left_monitor = StateMonitor(left_neuron, 'v', record=0)
left_event = EventMonitor(left, event='spike', variables='v')
right_monitor = StateMonitor(right_neuron, 'v', record=0)
right_event = EventMonitor(right, event='spike', variables='v')
#L_right = StateMonitor(inh_neurons, 'v', record=0)

print('Halfway there')





Halfway there


## Run the network

In [15]:
run(duration, report='text')

Starting simulation at t=0. s for a duration of 180. ms
180. ms (100%) simulated in 3s


## Evaluate the result

In [ ]:
import copy
fig, ax = plt.subplots()
ax.plot(data['times'], data['spikes'])

ax.set(xlabel='time (ms)', ylabel='Neuron index',
       title='Spikegen')
ax.grid()

fig.savefig("test.png")
plt.show()


figure()
subplot(211)
plot(left_monitor.t/ms, left_monitor.v[0])
xlabel('Time (ms)')
ylabel('v');
subplot(212)
plot(right_monitor.t/ms, right_monitor.v[0])
xlabel('Time (ms)')
ylabel('v');

print(left_event.it[1].__getitem__(0))
left_array = left_event.it[1].__array__()
right_array = right_event.it[1].__array__()
result = network_guess(left_array,right_array)

final_result = copy.deepcopy(result)
for i in range(0, len(result)):
    final_result[i] = (result[i] == labels[i])

print(final_result)

print(left_array)
print(right_array)


